Importing all the possible packages needed to scrape a website and to write the data to a pandas dataframe.

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import lxml.html as lh

The code below scrapes the website looking specifically for the table.  Once the table is found, all the elements within the table are transfered over into a local document. The headers of the table are separated from the data and each row is populated which can then be converted into a pandas dataframe. 

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')
col=[]
i=0
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    col.append((name,[]))
for j in range(1,len(tr_elements)):
    T=tr_elements[j]
    if len(T)!=3:
        break
    i=0
    for t in T.iterchildren():
        data=t.text_content() 
        if i>0:
            try:
                data=int(data)
            except:
                pass
        col[i][1].append(data)
        i+=1
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

The code below take the column Borough and finds all the values that are 'Not assigned' in the column.  Each row with this tag will be removed from the dataframe.  The head command is to ensure the rows were droped.  

In [3]:
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n


Output the number of rows and columns. 

In [4]:
df.shape

(211, 3)

In [5]:
data = pd.read_csv('https://cocl.us/Geospatial_data')
df2 = data
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
df2.rename(columns={'Postal Code':'Postcode'}, inplace=True)
df2.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
merge = pd.merge(df, df2, on=['Postcode'], how='inner')
merge.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods\n,43.753259,-79.329656
1,M4A,North York,Victoria Village\n,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront\n,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park\n,43.654260,-79.360636
4,M6A,North York,Lawrence Heights\n,43.718518,-79.464763
